In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)

In [9]:
os.chdir('/Users/jed/Documents/JED/Dataiku/drive-download-20250106T224222Z-001')
!ls

census_income_additional_info.pdf census_income_metadata.txt
census_income_learn.csv           census_income_test.csv


In [10]:
#load the train and test datasets into pandas df
train_data=pd.read_csv('census_income_learn.csv', header=None)
test_data=pd.read_csv('census_income_test.csv', header=None)
